In [1]:
import numpy as np
import torch
import string
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack
import math
import torch.utils.data as data
import json
import os
import pandas as pd
import random
import copy
import torch.utils.data.sampler as sampler
import visdom
import torch.optim.lr_scheduler as lr_scheduler
import pickle


In [92]:
import torch.utils.data as data

class Mul_data(data.Dataset):
    def __init__(self,d_type):
        self.d_type=d_type
        with open('../data/chat_frames.pickle',"rb") as f1:  
            self.chat_result=pickle.load(f1)
        with open('../data/video_frames.pickle',"rb") as f2:  
            self.image_result=pickle.load(f2)
        with open('../data/audio_frames.pickle',"rb") as f3:  
            self.audio_result=pickle.load(f3)
        with open('../label/label.pickle',"rb") as f4:  
            self.real_result=pickle.load(f4)
            
        if d_type=='train':
            self.sample =[102844235753356742, 102844294666422466, 102844341906256529, 102979081290790284, 102844212430271695, 102844401155937960, 102844412704496937, 102844412722519367, 102844235749031358, 102844412709674293, 102844341902586509, 102844235750997440, 102844212431975640, 102844294667995333, 102844341912220311, 102844235747261881, 102844294670551241, 102844412707708209, 102844212430599377, 102844224145685626, 102844401152857762, 102844341908026005, 102844341909598870, 102844283023599703, 102844235753749959, 102844294670026952, 102844212430075086, 102844283027531868, 102844212430927059, 102844212429419722, 102844283025696858, 102844224146472059, 102844412712164667, 102844401153971877, 102844235748310460, 102844412711116088, 102844235752111555, 102844283020453971, 102844294666881219, 102844401152267937, 102844212429944013, 102844294671796427, 102844401151874719, 102844412723567946, 102844294669568199, 102844412709346612, 102844212431779031, 102844212430402768, 102844412711443769, 102844283027925085, 102844235746868664, 102844283023206486, 102844401154168486, 102844212429550795, 102844341907370644, 102844412721339716, 102844294667405508, 102844294674876621, 102844212429288649, 102844401154430631]
        if d_type=='val':
            self.sample = [102844412705545516, 102844401153447587, 102844341912679064, 102844212431320277, 102844212431516886, 102844401152071328, 102844341906977427, 102844212429747404, 102844235747982779, 102844412716686654, 102844294669109446, 102844412722847048, 102844412721012035, 102844401151219358, 102844235746475447, 102844412720618818, 102904869420860038, 102910307641576395, 102844235751783874, 102844224146930812, 102844341905011343, 102844235752963525, 102844235748703677, 102844235749424575, 102844412705217835, 102844412723174729, 102844401153578660, 102844341905404560, 102844412706987311, 102844341906649746]
        if d_type=='test':
            self.sample = [102844412717014335, 102844224148503678, 102844412711836986, 102844401156069033, 102844294670878922, 102844341904683662, 102844412704890154, 102844235746082230, 102844224148896895, 102844294674286796, 102844412708953395, 102844235751390657, 102844212428895431, 102844412717407552, 102844212429092040, 102844412707380528, 102844212430730450, 102844224147717245, 102844412716227901, 102844212431058132, 102844412710001974, 102844412706659630]
            
        self.sum=np.insert(np.cumsum([len(self.chat_result[str(i)]) for i in self.sample]),0,0)
        print("data load fin")

        
    def __len__(self):
        return self.sum[-1]
    def __getitem__(self,index):
            vid=np.histogram(index,self.sum)#sum으로 누적으로 히스토그램이 깔려있음/ 그중에 index의 위치
            vid = np.where(vid[0]>0)[0][0]#몇번째 game을 쓸지!
            vframe=index-self.sum[vid]#그 게임 안에서의 몇번째 프레임인지
            game_id=str(self.sample[vid])

            window=[]#batch*7(window size)*3(highlight result)
            for idx in range(7): #7 : window size
                s_window=[]
                if vframe+idx<len(self.chat_result[game_id]):
                    s_window.append(float(self.chat_result[game_id][vframe]))#vframe의 chat
                    s_window.append(self.audio_result[game_id][vframe])#vframe의 audio
                    s_window.append(self.image_result[game_id][vframe])#vframe의 image
                else:
                    s_window=[0,0,0]#padding value
                window.append(s_window)


            label=int(self.real_result[game_id][vframe])
            return game_id,np.array(window),label

In [85]:
train=Mul_data('train')
val=Mul_data('val')
print(train[100])
train_loader=torch.utils.data.DataLoader(train,batch_size=128)
val_loader=torch.utils.data.DataLoader(val,batch_size=128)

data load fin
data load fin
('102844235753356742', array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]]), 0)


In [86]:
input_size=3
hidden_size=7
length=7
num_layers=3
class LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        self._clf1 = nn.LSTM(input_size, hidden_size,3)
        self._lin = nn.Sequential(nn.Linear(hidden_size, 2))

    def forward(self, x):
        hidden = Variable(torch.zeros(num_layers,length,hidden_size)) # (num_layers * num_directions, batch, hidden_size)
        cell = Variable(torch.zeros(num_layers,length,hidden_size)) # (num_layers * num_directions, batch, hidden_size)        out,hidden = self._clf1(x,h0)
        out,hidden = self._clf1(x,(hidden,cell))#batch*7*3
        out = self._lin(out[:,-1,:])
        return out

In [98]:
model=LSTM()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), 0.01,momentum=0.9,weight_decay=1e-4)


In [88]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
weight_dir='./result/'
if not os.path.exists(weight_dir):
    os.makedirs(weight_dir)
with open(weight_dir+'train_result','a') as f:
    f.write('=====result=======\n')
for epoch in range(200):
    losses = AverageMeter()
    top1 = AverageMeter()
    print(epoch)
    model.train()
    for i, (g,inputs,labels) in enumerate(train_loader):
        inputs=inputs.float()
        labels=labels
        optimizer.zero_grad()
        out=model(inputs)
        loss=criterion(out,labels)
        loss.backward()
        optimizer.step()
        
    model.eval()
    val_losses=AverageMeter()
    acc=0
    gt_sum=0
    with open(weight_dir+'train_result','a') as f:

        with torch.no_grad():
            for it, (g,inputs,labels) in enumerate(val_loader):
                inputs=inputs.float()
                labels=labels
                out=model(inputs)
                loss=criterion(out,labels)
                val_losses.update(loss,labels.size(0))
            print("epoch {} train_loss : {} , val_loss : {}".format(epoch,losses.avg,val_losses.avg))
            f.write("epoch {} train_loss : {} , val_loss : {}\n".format(epoch,losses.avg,val_losses.avg))
            torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train"))







0
epoch 0 train_loss : 0 , val_loss : 0.3642890453338623
1


In [114]:
test=Mul_data('test')
test_loader=torch.utils.data.DataLoader(test,batch_size=32)
dataset='./result/193train'
checkpoint=torch.load(dataset,map_location='cuda:0')
model.load_state_dict(checkpoint)
model.eval()
pred_sum = 0#model output
gt_sum = 0#label
tp_sum=0
fp_sum=0
fn_sum=0
acc=0
sum=0
result={}
with torch.no_grad():
    for it, (game_id,inputs,labels) in enumerate(test_loader):
        inputs=inputs.float()
        labels=labels
        output=model(inputs)
        TP,FP,TN,FN,pred_len, gt_len,pred=fmeasure(output.cpu(),labels.cpu())
        for idx,g in enumerate(game_id):
            if g not in result.keys():
                result[g]=pred[idx].tolist()
            else:
                result[g]+=pred[idx].tolist()
        print(TP,FP,TN,FN,pred_len, gt_len)
        tp_sum += TP
        fp_sum += FP
        fn_sum += FN
        pred_sum += pred_len
        gt_sum += gt_len
        acc=acc+TP+TN
        sum+=len(output)
    with open(weight_dir+'/train_result','a') as f:
        if tp_sum>0 and fp_sum>0 and fn_sum>0:
            precision = tp_sum/(tp_sum+fp_sum)
            recall = tp_sum / (tp_sum+fn_sum)
            f1 = (2*precision*recall / (precision + recall)) * 100
            accuracy=acc/sum
            print( tp_sum, fp_sum, fn_sum)
            print('[{}/{}], prec:{}, recall:{}, f1:{}, acc: {}'.format(it, len(val_loader), precision, recall, f1,accuracy))
            f.write('{}, prec:{}, recall:{}, f1:{}, acc : {}\n'.format(dataset, precision, recall, f1,accuracy))

data load fin
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 31 1 tensor(0) tensor(1)
15 4 11 2 tensor(19) tensor(17)
6 0 26 0 tensor(6) tensor(6)
19 5 2 6 tensor(24) tensor(25)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
12 3 11 6 tensor(15) tensor(18)
0 0 32 0 tensor(0) tensor(0)
3 0 25 4 tensor(3) tensor(7)
1 10 21 0 tensor(11) tensor(1)
16 5 11 0 tensor(21) tensor(16)
10 0 21 1 tensor(10) tensor(11)
29 3 0 0 tensor(32) tensor(29)
0 12 20 0 tensor(12) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 8 24 0 tensor(8) tensor(0)
0 16 16 0 tensor(16) tensor(0)
0 0 32 0 tensor(0) tensor(0)
13 0 8 11 tensor(13) tensor(24)
11 5

0 15 17 0 tensor(15) tensor(0)
0 6 26 0 tensor(6) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 25 7 tensor(0) tensor(7)
8 0 3 21 tensor(8) tensor(29)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 31 1 tensor(0) tensor(1)
16 0 0 16 tensor(16) tensor(32)
3 3 21 5 tensor(6) tensor(8)
0 0 0 32 tensor(0) tensor(32)
0 0 23 9 tensor(0) tensor(9)
0 0 32 0 tensor(0) tensor(0)
9 0 10 13 tensor(9) tensor(22)
17 5 5 5 tensor(22) tensor(22)
0 0 27 5 tensor(0) tensor(5)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
13 0 11 8 tensor(13) tensor(21)
15 0 10 7 tensor(15) tensor(22)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
5 0 24 3 tensor(5) tensor(8)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(

0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
1 5 26 0 tensor(6) tensor(1)
15 7 10 0 tensor(22) tensor(15)
0 2 30 0 tensor(2) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
13 0 19 0 tensor(13) tensor(13)
3 3 25 1 tensor(6) tensor(4)
0 7 25 0 tensor(7) tensor(0)
5 0 22 5 tensor(5) tensor(10)
0 0 32 0 tensor(0) tensor(0)
0 0 28 4 tensor(0) tensor(4)
14 5 9 4 tensor(19) tensor(18)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
11 0 17 4 tensor(11) tensor(15)
7 14 3 8 tensor(21) tensor(15)
0 1 31 0 tensor(1) tensor(0)
0 0 32 0 tensor(0) tensor(0)
6 1 25 0 tensor(7) tensor(6)
8 4 19 1 tensor(12) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 2 30 0 tensor(2) tensor(0)
0 4 11 17 tensor(4) tensor(17)
21 0 0 11 tensor(21) tensor(32)
3 4 25 0 tensor(7) tens

0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
23 2 4 3 tensor(25) tensor(26)
0 12 20 0 tensor(12) tensor(0)
0 10 22 0 tensor(10) tensor(0)
0 0 32 0 tensor(0) tensor(0)
28 0 3 1 tensor(28) tensor(29)
7 5 20 0 tensor(12) tensor(7)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
13 9 6 4 tensor(22) tensor(17)
0 8 24 0 tensor(8) tensor(0)
0 0 32 0 tensor(0) tensor(0)
8 0 24 0 tensor(8) tensor(8)
8 5 19 0 tensor(13) tensor(8)
0 8 24 0 tensor(8) tensor(0)
0 8 24 0 tensor(8) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 15 17 tensor(0) tensor(17)
0 0 7 25 tensor(0) tensor(25)
9 2 11 10 tensor(11) tensor(19)
6 4 22 0 tensor(10) tensor(6)
0 0 32 0 tensor(0) tensor(0)
0 0 14 18 tensor(0) tensor(18)
0 0 32 0 tensor(0) tensor(0)
0 9 23 0 tensor(9) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 17 15 tensor(0) tensor(15)
14 0 7 11 tensor(14) tensor(25)
31 0 0 1 tensor(31) tensor(32)
0 0 32 0 tensor

0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
15 7 9 1 tensor(22) tensor(16)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
11 0 15 6 tensor(11) tensor(17)
5 5 22 0 tensor(10) tensor(5)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
5 4 23 0 tensor(9) tensor(5)
22 7 0 3 tensor(29) tensor(25)
4 2 26 0 tensor(6) tensor(4)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
8 7 12 5 tensor(15) tensor(13)
4 5 23 0 tensor(9) tensor(4)
0 0 32 0 tensor(0) tensor(0)
5 3 19 5 tensor(8) tensor(10)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 9 23 0 tensor(9) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
12 15 2 3 tensor(27) tensor(15)
0 9 23 0 tensor(9) tensor(0)
0 5 27 0 tensor(5) tensor(0)
12 5 12 3 tensor(17) tensor(15)
1 3 27 1 tensor(4) tensor(2)
0 0 32 0 tensor(0) tensor(

In [100]:
def fmeasure(output, target):
    _, pred = output.topk(1, 1, True, True)
    pred = pred.view(-1,1)
    target = target.view(-1,1)

    #overlap = ((pred== 1) + (target == 1)).gt(1)
    #overlap = overlap.view(-1,1)
    TP = len(np.where((pred==1)&(target==1)==True)[0]) # True positive
    FP = len(np.where((pred==1)&(target==0)==True)[0]) # Condition positive = TP + FN
    TN = len(np.where((pred==0)&(target==0)==True)[0])
    FN = len(np.where((pred==0)&(target==1)==True)[0])

    
    #overlap_len = overlap.data.long().sum()
    pred_len = pred.data.long().sum()
    gt_len   =  target.data.long().sum()

    return TP,FP,TN,FN,pred_len, gt_len,pred

In [9]:
b=torch.rand((1,2,3))

In [10]:
a=torch.transpose(b,1,2)

In [11]:
a

tensor([[[0.1380, 0.0111],
         [0.8294, 0.4059],
         [0.0521, 0.1911]]])

In [12]:
b

tensor([[[0.1380, 0.8294, 0.0521],
         [0.0111, 0.4059, 0.1911]]])